# Programming in Python for Data Science - Final Project
### *by Maria Carolina Cunha Netto*

# Exploratory data analysis of the Disney datasets

This notebook will be showing some exploratory data analysis for the `Disney Character Success` dataset located [here](https://data.world/kgarrett/disney-character-success-00-16).


# Introduction

## Question(s) of interests
In this analysis, I will be investigating questions associated with the collection of Disney datasets.

I am interested in finding out which genre is the most and the least successful over the years, which director produced the most profitable movies and the money-making decades. This is interesting because the Walt Disney Company was not focused in only one style, they produced a large variety of film genres. 

I would expect the genre of **Western** movies being the least popular and therefore the least profitable. The gem are probably the **Musicals**. Most likely the director that made more profit worked with Musicals. I am not sure about the money-making decades but I am excited to find out.


## Dataset description 

The below descriptions were taken directly from the [website](https://data.world/kgarrett/disney-character-success-00-16) where the data was obtained.

The "Disney Character Success" dataset is composed of 5 tables, `disney-characters.csv`, `disney-director.csv`, `disney-voice-actors.csv`, `disney_revenue_1991-2016.csv` and `disney_movies_total_gross.csv`. Each table is stored in a `.csv` file and contains different information about Disney characters, box office success & annual gross income. I will be using the `disney_movies_total_gross` and `disney-director` tables formally described below:

* **disney_movies_total_gross.csv**
    * This file contains information about Disney movie box office gross and inflation adjustments. It includes the movie title, release date, genre, mpaa_rating, total gross, and inflation adjusted gross.
    
* **disney-director.csv**
    * This file contains information on Disney directors, including the name of the film and the director. Note we will assume that each movie has only one director.

# Methods and Results

Since I am only interested in computing data related to genres of the movie titles, total gross, directors and the release dates, I will probably only use the tables that contain these information, such as **disney_movies_total_gross** and **disney-director**.

Before moving further, let's import the tables and do some basic visualizations.

In [1]:
# Import libraries needed for this project
from fp_functions import movie_cleaner_str
from fp_functions import director_cleaner_str
from fp_functions import list_repeted
import pandas as pd
import numpy as np
import altair as alt
import pytest

# Import the required files
#characters = pd.read_csv('data/disney-characters.csv')
#voice_actors = pd.read_csv('data/disney-voice-actors.csv')
#revenue = pd.read_csv('data/disney_revenue_1991-2016.csv')
movies = pd.read_csv('data/disney_movies_total_gross.csv', parse_dates=['release_date'])
directors = pd.read_csv('data/disney-director.csv')

In [2]:
# Formatting the functions file
!black fp_functions.py
!flake8 fp_functions.py

All done! ✨ 🍰 ✨
1 file left unchanged.


In [3]:
# Check my tests on the file fp_tests.py using pytest
!pytest fp_tests.py

============================= test session starts ==============================
platform linux -- Python 3.8.5, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: /home/jupyter/prog-python-ds-students/release/final_project
plugins: anyio-3.2.1, dash-1.20.0
collected 2 items                                                              

fp_tests.py ..                                                           [100%]

============================== 2 passed in 0.79s ===============================


After importing the files needed, let's see what the tables look like.

In [ ]:
directors.head().style.set_caption("\nTable 1 - Directors (first 5 rows)")

In [ ]:
directors.info()

The table **directors** has 56 rows with 2 columns. No missing values. *Name* is the movie title and each one has its director associated. In the column *director*, the movie "Fantasia" for example has a supposed error. It just mentions 'full credits' instead of the name of the director. We will leave it as it is for now.

In [ ]:
directors[directors['director']=="full credits"]

In [ ]:
movies.head().style.set_caption("\nTable 2 - Movies (first 5 rows)")

In [ ]:
movies.info()

The table **movies** has 579 rows and 6 columns. Every movie title has a release date, the genre, the Motion Picture Association Of America (MPAA) rating, the total gross and the inflation adjusted gross. In this table we have some missing values in the columns *genre* and *MPAA*.

Now that we have an idea of the information in each table, let's use the table **movies** to look at the average inflation adjusted gross per movie genre. 

The first step is cleaning the data and preparing for the analysis.

In [ ]:
# Cleaning the table movies and saving as "movies2"
movies2 = movie_cleaner_str(movies)

It is important to verify if there is any movie title that appears more than once in the data frame. If it is happening, it might give us a biased result. Let's check that before moving on.

In [ ]:
# Is there any movie_title repeated in this DataFrame?
movies2['movie_title'].size - pd.unique(pd.Series(movies2['movie_title'])).size

for i in list_repeted(movies2['movie_title']):
    print(movies2[movies2['movie_title']==i])

There are 5 movies repeated in this DataFrame:

* The comedy "101 Dalmatians" was released in 1961 and 1996.

* "Bad Company" was released first in 1995 as a "Thriller/Suspense" and later in 2002 classified as "Action".

* The drama "Cinderella" was released in 1950 and also in 2015.

* According to the dates we have, the comedy "Freaky Friday" was released in 1977 and also in 2003. Note that as we are not interested in the *MPAA_rating* we will ignore this missing value for now.

* "The Jungle Book" was released 3 times: in 1967 as a "Musical", and in 1994 and 2016 classified as "Adventure".

Admitting there are remakes and it is possible to have more than one release date for the same movie title, we will keep all those rows untouched.
Let's check if there is any abnormality in the *total_gross* and *inflation_adjusted_gross* columns.

In [ ]:
movies2.agg({
        "total_gross": ["min", "max", "mean", "count"],
        "inflation_adjusted_gross": ["min", "max", "mean", "count"],
    }).apply(lambda s: s.apply('{:0,.0f}'.format)).style.set_caption("\nTable 3 - Descriptive Statistics: Total Gross & Inflation Adjusted Gross")

In [ ]:
# Selecting only movies with total gross < $10,000
movies2[movies2['inflation_adjusted_gross']<10000].style.set_caption("\nTable 4 - Movie Titles with Total Gross < $10,000")

Selecting only movies with total gross less than \\$10,000 we find the top 5 biggest box-office bombs or disasters of Disney production. 

Since we are also interested in some analysis considering the genre of the movies, we will delete the rows with missing values in the column *genre*.

In [ ]:
# Deleting the rows with missing values in 'genre'
movies2 = movies2.dropna(subset=['genre'])
movies2.info()

We now have 562 rows. 

The column *release_date* looks accurate because we cannot spot any problem with it. We can see that "Snow White and the Seven Dwarfs", was Disney's first Musical, released in 1937 and the newest movie released (included in this table **movies**) was the adventure "Rogue One: A Star Wars Story" in 2016 as we can see below in Table 5.

In [ ]:
movies2[(movies2['release_date']==movies2['release_date'].min()) | (movies2['release_date']==movies2['release_date'].max())].style.set_caption("\nTable 5 - First & Last Movies Released")


Now that the table **movies** is clean and verified, I will go back to the initial analysis of interest and group the table by genre to compute the average inflation_adjusted_gross.

In [ ]:
# Group by genre and compute the average of inflation_adjusted_gross.
gross_by_genre = movies2.groupby('genre').agg({'inflation_adjusted_gross':'mean'}).sort_values(by ='inflation_adjusted_gross', ascending=False).reset_index()

In [ ]:
# Using altair to generate a bar plot
genre_plot = (
    alt.Chart(gross_by_genre, width=500, height=300)
    .mark_bar(color='black')
    .encode(
        x=alt.X("genre:O", title="Genre",sort="-y"),
        y=alt.Y("inflation_adjusted_gross:Q", title="Inflation Adjusted Gross ($)"),
    )
    .properties(title="Figure 1 - Average Inflation Adjusted Gross by Genre")
)
genre_plot

From Figure 1, it is clear that the most profitable genre over the years is Musical. It was a surprise for me that Disney also produces Documentaries and though predictably they are the least profitable for the company. Among the 12 genres mentioned in the table **movies**, only Musical, Adventure and Action made an average of inflation adjusted gross over \\$100,000,000.
We will aggregate the least profitable genres into a new category "Others" to make it easier to visualize the 3 most profitable Disney genres.

In [ ]:
movies3 = movies2.assign(genre2 = movies2['genre'])
movies3.loc[ ((movies3['genre'] != 'Musical') & (movies3['genre'] != 'Adventure') & (movies3['genre'] != 'Action')), 'genre2'] = 'Others'

# Group by movie_title and compute the average of inflation_adjusted_gross.
gross_by_genre2 = movies3.groupby('genre2').agg({"inflation_adjusted_gross":'mean'}).sort_values(by ='inflation_adjusted_gross', ascending=False).reset_index()

In [ ]:
# Using altair to generate a bar plot
genre_plot = (
    alt.Chart(gross_by_genre2, width=300, height=200)
    .mark_bar(color = 'black')
    .encode(
        x=alt.X("genre2:O", title="Genre",sort="-y"),
        y=alt.Y("inflation_adjusted_gross:Q", title="Inflation Adjusted Gross ($)"),
    )
    .properties(title=["Figure 2 - Disney Success:","Average Inflation Adjusted Gross by Genre"])
)
genre_plot

To find out which director produced the most profitable movie, we need to create a new dataframe that contains information from both tables **movies** and **directors**.

In [ ]:
# Cleaning the table directors and saving as "directors2"
directors2 = director_cleaner_str(directors)

In [ ]:
# Merging both DataFrames movies & directors
directors2 = directors2.rename(columns={'name': 'movie_title'})
movies_directors_merged = pd.merge(directors2, movies3, on="movie_title", how='inner')
movies_directors_merged.style.set_caption("\nTable 6 - Merged DataFrame Movies & Directors")

Table 6 is the newly merged DataFrame and contains 47 rows. We decided to keep oly the rows with coincident information in both tables **movies** and **directors** at the same time. It would not make sense to keep a row only with the name of the movie and the director, for example. 

There is, however, an issue with the new DataFrame created. We explored before that some of the movies were released more than once. Bringing the information of the name of the director does not guarantee which was the version of the movie we should be considering for that director. Taking as an example the movie “Cinderella”, printed above (Table 6: rows 3-4) the first release date was 1950 and then again in 2015. It is not impossible but surely not likely that it had been the same director both times. When we merged the tables into this new one, only the movie title was considered, so "Wilfred Jackson" was added as the director of "Cinderella" in both rows.

We can compare the list of repeated movie titles from the table **directors** to the list of repeated directors in the newly merged table to see if they actually had been working more than once in the same title (different release dates).

In [ ]:
print("Number of repeated movie titles in the *directors* table:", len(list_repeted(directors2['movie_title']))) 

Since there are no repeat movie titles in the table **directors**, and we know that some directors worked in more than one movie (apparently not the same movie), we still cannot assume which was the version they worked in. For exemple, according to Wikipedia, "Cinderella" was directed by Wilfred Jackson only in 1950, in 2015 it was a different director. 

The database provided will limit the result to the title, we will not be able to specify when the movie was released. We can only find out which director produced the most profitable movie, which is enough for this particular analysis. 

We can now compute the average inflation adjusted gross by director.

In [ ]:
# Group by movie_title and compute the average of inflation_adjusted_gross.
gross_by_director = movies_directors_merged.groupby('director').agg({"inflation_adjusted_gross":'mean'}).sort_values(by ='inflation_adjusted_gross', ascending=False).reset_index()

In [ ]:
# Using altair to generate a bar plot
director_plot = (
    alt.Chart(gross_by_director, width=550, height=300)
    .mark_bar(color='black')
    .encode(
        x=alt.X("director:O", title="Director",sort="-y"),
        y=alt.Y("inflation_adjusted_gross:Q", title="Inflation Adjusted Gross ($)"),
    )
    .properties(title="Figure 3 - Average Inflation Adjusted Gross by Director")
)
director_plot

In [ ]:
movies_directors_merged[(movies_directors_merged['director']=='David Hand') | (movies_directors_merged['director']=='Ted Berman')].style.set_caption("\nTable 7 - Most & Least Profitable Directors")

Not surprisingly, the most profit-making director worked in a Musical, the most profitable genre for Disney. The movie he worked in was "Snow White And The Seven Dwarfs", released in 1937 (Table 7). On the other hand, Figure 2 shows that "Adventure" is the second most profitable genre for Disney, and Ted Berman was unsuccessful directing the adventure "The Black Cauldron", released in 1985. 

In the last analysis we will explore if there was any trend in Disney success. Because the merged table has only 47 rows, we will use again the table **movies** that contain all the movies and their release dates.


In [ ]:
movies3 = movies3.assign(year = movies3['release_date'].dt.year)
# Count movie_titles by Year.
movies_by_year = movies3.groupby('year').agg({"release_date":'count', "inflation_adjusted_gross":'mean'}).reset_index()

In [ ]:
# Using altair to generate a bar plot
movies_by_year_plot = (
    alt.Chart(movies_by_year, width=700, height=400)
    .mark_bar(color='black')
    .encode(
        x=alt.X("year:O", title="Year"),
        y=alt.Y("release_date:Q", title="Count of Movie Titles"),
    )
    .properties(title="Figure 4 - Number of Movies Released by Year")
)
movies_by_year_plot

From Figure 4 we can see that in 1937 "Snow White And The Seven Dwarfs" was the only movie released. Disney increased the number of movies produced along the years with a boom in the 90's.
We will aggregate the years into decades to make it easier to visualize it.

In [ ]:
# Create a column for the decate based on the Year
movies_by_dec = movies_by_year.assign(decade = movies_by_year['year'] - (movies_by_year['year'])%10) 

# Using altair to generate a bar plot
movies_by_dec_plot = (
    alt.Chart(movies_by_dec, width=500, height=300)
    .mark_bar(color='black')
    .encode(
        x=alt.X("decade:O", title="Decade"),
        y=alt.Y("release_date:Q", title="Count of Movie Titles"),
    )
    .properties(title="Figure 5 - Number of Movies Released by Decade")
)
movies_by_dec_plot


We already know from Figure 5 that from 1990-2010 was the period that Disney produced more movies. Now last but not least, we want to see which were the money-making decades.

In [ ]:
movies_by_dec = movies_by_year.assign(decade = movies_by_year['year'] - (movies_by_year['year'])%10) 

# Using altair to generate a bar plot
gross_by_dec_plot = (
    alt.Chart(movies_by_dec, width=500, height=300)
    .mark_bar(color='black')
    .encode(
        x=alt.X("decade:O", title="Decade"),
        y=alt.Y("inflation_adjusted_gross:Q", title="Average inflation adjusted gross ($)"),
    )
    .properties(title="Figure 6 - Average Inflation Adjusted Gross by Decade")
)
gross_by_dec_plot


In Figure 6, we see that the first decade was the most successful for the company and there was a big decline afterwards. There was no similar success again.

# Discussions

In this project, I analyzed the Disney datasets and tried to present which movie genres were more successful and also which directors were the biggest success and box-office disasters. 

After some exploratory data analysis, as expected, "Musicals" are the big hit for Disney, mainly due to the major success "Snow White And The Seven Dwarfs" in 1937. The adventure "The Black Cauldron" directed by Ted Berman could probably be considered a box-office bomb. 

From Figure 3, the director that brought more money to Disney was David Hand, with over \\$5,000,000,000. He worked on the musical "Snow White And The Seven Dwarfs".

Disney increased the number of productions over the years but it did not reflect in their profit. One possibility might be the competition that probably increased over the years and a more demanding audience.

One great movie is not enough, however Disney has a diverse range of businesses and their profit is not only associated with movies. Another question that could be looked at would be where the company is investing more to keep their business innovative and profitable nowadays.

# References

Most of the codes used in this notebook were copied from my own notes from the course 'Programming in Python for Data Science' from UBC Extended Learning. Some other parts were borrowed from online resources. I take no credit for parts that are not mine.

## Resources used
* [Data Source](https://data.world/kgarrett/disney-character-success-00-16)
    * These Disney databases used in this work were curated by **Kelly Garrett**.
* [Coding](https://stackoverflow.com)
    * Some of the codes were inspired by **Stackoverflow** users:
    * [How to ignore comments on flake8](https://stackoverflow.com/questions/47876079/how-to-tell-flake8-to-ignore-comments#:~:text=Using%20an%20inline%20comment%20%23%20noqa,Long%20url%20as%20a%20reference.)
    * [How to list duplicates](https://stackoverflow.com/questions/9835762/how-do-i-find-the-duplicates-in-a-list-and-create-another-list-with-them)
    * [How to format the result from .describe()](https://stackoverflow.com/questions/40347689/dataframe-describe-suppress-scientific-notation)
    * [How to add table title](https://stackoverflow.com/questions/57958432/how-to-add-table-title-in-python-preferably-with-pandas)
* [Wikipedia: Cinderella 1950](https://en.wikipedia.org/wiki/Cinderella_(1950_film))
    * Inspiration to write about the movies.